In [5]:
import os
import copy
import wandb

from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)
from trl import (
    ModelConfig,
    PPOConfig,
    ScriptArguments,
    get_quantization_config,
    get_peft_config,
)
from peft import (
    PeftModelForSequenceClassification,
    TaskType, 
    get_peft_model,
)
from accelerate import PartialState

from fed_ppo.utils import PolicyCommutator, CustomPPOTrainer, frozen_copy

In [6]:
BASE_MODEL = "LLama-3.2-1B-Instruct"
NUM_EPOCHS = 3
NUM_AGENTS = 2
COMMUTANT  = [
    [0.8, 0.2],
    [0.2, 0.8]
]

In [7]:
os.environ["CUDA_DEVICE_ORDER"]     = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]  = "5,6"
os.environ["WANDB_PROJECT"]         = f"FedPPO-{BASE_MODEL}"
os.environ["WANDB_ENTITY"]          = "RADFAN"

In [5]:
###############################################################################
# Configs
###############################################################################

script_args = ScriptArguments(
    dataset_name        = "HuggingFaceH4/ultrachat_200k",
    dataset_train_split = "descriptiveness",
)

# Model
# =============================================================================

model_config = ModelConfig(
    model_name_or_path   = f"meta-llama/{BASE_MODEL}",
    # LoRA
    # ---------------------------------------------------------------------------------------------
    use_peft             = True,
    lora_r               = 8,
    lora_alpha           = 16,
    lora_dropout         = 0.0,
    lora_task_type       = TaskType.CAUSAL_LM,
    lora_target_modules  = ["q_proj", "k_proj", "v_proj", "o_proj"],
    # Quantization
    # ---------------------------------------------------------------------------------------------
    load_in_8bit         = False,
    load_in_4bit         = False,
    torch_dtype          = "bfloat16",
)

# Value model
# =================================================================================================

value_model_config = ModelConfig(
    # LoRA
    # ---------------------------------------------------------------------------------------------
    use_peft            = True,
    lora_r              = 8,
    lora_alpha          = 16,
    lora_dropout        = 0.0,
    lora_task_type      = TaskType.SEQ_CLS,
    lora_target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    # Quantization
    # ---------------------------------------------------------------------------------------------
    load_in_8bit        = False,
    load_in_4bit        = False,
    torch_dtype         = "bfloat16",
)

# Reward model
# =================================================================================================

reward_model_config = ModelConfig(
    model_name_or_path  = "RLHF-And-Friends/Llama-3.2-1B-Instruct-Reward-LoRA8r",
    use_peft            = True,
    load_in_8bit        = False,
    load_in_4bit        = False,
)

# PPO trainer config
# =================================================================================================

ppo_config = PPOConfig(
    # Common
    # ---------------------------------------------------------------------------------------------
    exp_name            = f"FedPPO-{BASE_MODEL}",
    output_dir          = f"FedPPO-{BASE_MODEL}",
    dataset_num_proc    = 16,
    num_mini_batches    = 1,
    learning_rate       = 1e-5,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 8,
    num_train_epochs    = 1,
    response_length     = 512,
    # Push to hub after training
    # ---------------------------------------------------------------------------------------------
    push_to_hub         = True,
    hub_model_id        =f"RLHF-And-Friends/FedPPO-{BASE_MODEL}",
    # On-policy params
    # ---------------------------------------------------------------------------------------------
    missing_eos_penalty = 1.0,
    local_rollout_forward_batch_size = 1,
    # PPO params
    # ---------------------------------------------------------------------------------------------
    num_ppo_epochs      = 1,
    whiten_rewards      = False,
    kl_coef             = 0.05,
    cliprange           = 0.2,
    vf_coef             = 0.1,
    cliprange_value     = 0.2,
    gamma               = 1.0,
    lam                 = 0.95,
)

# Distinct PPO configs
# =================================================================================================

ppo_configs = [copy.copy(ppo_config) for _ in range(NUM_AGENTS)]

for agent_idx, config in enumerate(ppo_configs):
    config.exp_name     = f"{config.exp_name}-A{agent_idx}"
    config.output_dir   = f"{config.output_dir}-A{agent_idx}"
    config.hub_model_id = f"{config.hub_model_id}-A{agent_idx}"


In [6]:
###############################################################################
#  Tokenizer
###############################################################################

tokenizer = AutoTokenizer.from_pretrained(
    model_config.model_name_or_path,
    use_fast = True,
)
tokenizer.add_special_tokens({"pad_token": "[PAD]"})


In [7]:
# Tokenizer
# -------------------------------------------------------------------------------------------------

tokenizer = AutoTokenizer.from_pretrained(
    model_config.model_name_or_path,
    use_fast = True,
)
tokenizer.add_special_tokens({"pad_token": "[PAD]"})

# SFT
# -------------------------------------------------------------------------------------------------

sft = AutoModelForCausalLM.from_pretrained(
    model_config.model_name_or_path,
    quantization_config = get_quantization_config(model_config),
    torch_dtype=model_config.torch_dtype,
)
sft.resize_token_embeddings(len(tokenizer), mean_resizing=False)
sft.config.pad_token_id = tokenizer.pad_token_id

# Policy Models
# -------------------------------------------------------------------------------------------------

policy_models = [
    get_peft_model(
        sft,
        get_peft_config(model_config)
    )
    for _ in range(NUM_AGENTS)
]

# Base Value Model
# -------------------------------------------------------------------------------------------------

base_value_model = AutoModelForSequenceClassification.from_pretrained(
    model_config.model_name_or_path,
    num_labels = 1,
    quantization_config = get_quantization_config(value_model_config),
    torch_dtype=model_config.torch_dtype,
)
base_value_model.resize_token_embeddings(len(tokenizer), mean_resizing=False)
base_value_model.config.pad_token_id = tokenizer.pad_token_id

# Reward model
# -------------------------------------------------------------------------------------------------

reward_model = PeftModelForSequenceClassification.from_pretrained(
    base_value_model,
    reward_model_config.model_name_or_path,
    num_labels = 1,
    quantization_config = get_quantization_config(reward_model_config)
)

# Value models
# -------------------------------------------------------------------------------------------------

value_models = [
    get_peft_model(
        base_value_model,
        get_peft_config(value_model_config)
    )
    for _ in range(NUM_AGENTS)
]


Some weights of GPTNeoXForSequenceClassification were not initialized from the model checkpoint at EleutherAI/pythia-70m-deduped and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPTNeoXForSequenceClassification were not initialized from the model checkpoint at EleutherAI/pythia-70m-deduped and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPTNeoXForSequenceClassification were not initialized from the model checkpoint at EleutherAI/pythia-70m-deduped and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
###############################################################################
#  Dataset
###############################################################################

dataset = load_dataset(
    script_args.dataset_name,
    split=script_args.dataset_train_split
)
eval_samples = 100
train_dataset = dataset.select(range(len(dataset) - eval_samples))
eval_dataset = dataset.select(range(len(dataset) - eval_samples, len(dataset)))
dataset_text_field = "prompt"

def prepare_dataset(dataset, tokenizer):
    """
    pre-tokenize the dataset before training; only collate during training
    """

    def tokenize(element):
        outputs = tokenizer(
            element[dataset_text_field],
            padding=False,
        )
        return {"input_ids": outputs["input_ids"]}

    return dataset.map(
        tokenize,
        batched=True,
        remove_columns=dataset.column_names
    )

# Compute that only on the main process for faster data processing.
# see: https://github.com/huggingface/trl/pull/1255
with PartialState().local_main_process_first():
    train_dataset = prepare_dataset(train_dataset, tokenizer)
    eval_dataset = prepare_dataset(eval_dataset, tokenizer)
    
train_datasets = [
    train_dataset.shard(num_shards = NUM_AGENTS, index = agent_idx)
    for agent_idx in range(NUM_AGENTS)
]

In [ ]:
###############################################################################
#  Training
###############################################################################

for epoch in range(NUM_EPOCHS):

    frozen_policies = [frozen_copy(model) for model in policy_models]

    reference_models = PolicyCommutator(
        policies = frozen_policies,
        commutant = COMMUTANT
    )
    for idx in range(NUM_AGENTS):
        trainer = CustomPPOTrainer(
            config            = ppo_configs[idx],
            processing_class  = tokenizer,
            policy            = policy_models[idx],
            ref_policy        = reference_models[idx],
            reward_model      = reward_model,
            value_model       = value_models[idx],
            train_dataset     = train_datasets[idx],
            eval_dataset      = eval_dataset,
        )
        trainer.train()
        wandb.finish()

        if ppo_configs[idx].push_to_hub:
            trainer.push_to_hub(dataset_name=script_args.dataset_name)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: arqa39 (RADFAN). Use `wandb login --relogin` to force relogin


===training policy===


From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


Step,Training Loss


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ She couldn't get the saw-player the kid had │                                              │ 2.3284964561462402 │
│ mentioned out of her mind. Sounds Hawaiian, │                                              │                    │
│ she thought over and over again as Eddie    │ "I'm sorry," Eddie said. "I'm sorry."        │                    │
│ pushed her grimly along in the new          │                                              │                    │
│ wheelchair, weaving in and out of the       │ "I'm sorry," Eddie said. "I'm sorry."        │                    │
│ stalled vehicles. Sounds Hawaiian, doesn't  │                                              │                    │
│ it? Sounds fucking Hawaiian, doesn't it.    │ "I'm sorry," Eddie said. "I'm sorry."        │                    │
│                                             │                                              │                    │
│                                             │ "I'm sorry," Eddie                           │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ "You little piss-ant!" the girl snapped.    │  "I don't know what to do."                  │ 3.765852928161621  │
│ "Don't tell me I slipped up. She died at    │                                              │                    │
│ seventeen. That's why I wasn't there. I was │ "I don't know what to do."                   │                    │
│ never notified."                            │                                              │                    │
│                                             │ "I don't know what to do."                   │                    │
│ "But I don't do sixteen," he said, his      │                                              │                    │
│ voice going nasty.                          │ "I don't know what to do."                   │                    │
│                                             │                                              │                    │
│                                             │ "I don't know what to do."                   │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Daniel flashed one of his own-a real one,   │  I was looking at the ceiling. I was looking │ 3.0170459747314453 │
│ this time.                                  │ at the ceiling. I was looking at the         │                    │
│                                             │ ceiling. I was looking at the ceiling. I was │                    │
│ "I almost had a heart attack when Mom       │ looking at the ceiling. I was looking at the │                    │
│ almost had a heart attack," he said, his    │ ceiling. I was looking at the ceiling. I was │                    │
│ voice quiet. Serious. "I'm-I'm happy you're │ looking at                                   │                    │
│ okay."                                      │                                              │                    │
│                                             │                                              │                    │
│ I looked around the room.                   │                                              │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ "Lights," the Oracle announced.  Images     │  I'm not sure what to do."                   │ 3.4875550270080566 │
│ sprang up on the panels, showing coral and  │ "I'm not

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ She couldn't get the saw-player the kid had │                                              │ 4.0301432609558105 │
│ mentioned out of her mind. Sounds Hawaiian, │                                              │                    │
│ she thought over and over again as Eddie    │ "I'm sorry," she said.                       │                    │
│ pushed her grimly along in the new          │                                              │                    │
│ wheelchair, weaving in and out of the       │ "I'm sorry," she said.                       │                    │
│ stalled vehicles. Sounds Hawaiian, doesn't  │                                              │                    │
│ it? Sounds fucking Hawaiian, doesn't it.    │ "I'm sorry," she said.                       │                    │
│                                             │                                              │                    │
│                                             │ "I'm sorry," she said.                       │                    │
│                                             │                                              │                    │
│                                             │ "I'm sorry," she said.                       │                    │
│                                             │                                              │                    │
│                                             │ "                                            │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ "You little piss-ant!" the girl snapped.    │                                              │ 3.4986019134521484 │
│ "Don't tell me I slipped up. She died at    │                                              │                    │
│ seventeen. That's why I wasn't there. I was │ "I'm not," she said. "I'm not."              │                    │
│ never notified."                            │                                              │                    │
│                                             │ "I'm not," he said. "I'm not."               │                    │
│ "But I don't do sixteen," he said, his      │                                              │                    │
│ voice going nasty.                          │ "I'm not," she said.                         │                    │
│                                             │                                              │                    │
│                                             │ "I'm not," he said.                          │                    │
│                                             │                                              │                    │
│                                             │ "                                            │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Daniel flashed one of his own-a real one,   │  I was in the middle of a room. I was in the │ 3.7285637855529785 │
│ this time.                                  │ middle of a room. I was in the middle of a   │                    │
│                                             │ room. I was in the middle of a room. I was   │                    │
│ "I almost had a heart attack when Mom       │ in the middle of a room.                     │                    │
│ almost had a heart attack," he said, his    │                                              │                    │
│ voice quiet. Serious. "I'm-I'm happy you're │ "I'm sor

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ She couldn't get the saw-player the kid had │                                              │ 3.488396406173706  │
│ mentioned out of her mind. Sounds Hawaiian, │                                              │                    │
│ she thought over and over again as Eddie    │ "I'm not a kid," she said.                   │                    │
│ pushed her grimly along in the new          │                                              │                    │
│ wheelchair, weaving in and out of the       │ "I'm not a kid," she said.                   │                    │
│ stalled vehicles. Sounds Hawaiian, doesn't  │                                              │                    │
│ it? Sounds fucking Hawaiian, doesn't it.    │ "I'm not a kid," she said.                   │                    │
│                                             │                                              │                    │
│                                             │ "I'm not a kid," she said.                   │                    │
│                                             │                                              │                    │
│                                             │ "I'm                                         │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ "You little piss-ant!" the girl snapped.    │                                              │ 3.6105432510375977 │
│ "Don't tell me I slipped up. She died at    │                                              │                    │
│ seventeen. That's why I wasn't there. I was │ "I'm not," she said.                         │                    │
│ never notified."                            │                                              │                    │
│                                             │ "I'm not," he said.                          │                    │
│ "But I don't do sixteen," he said, his      │                                              │                    │
│ voice going nasty.                          │ "I'm not," she said.                         │                    │
│                                             │                                              │                    │
│                                             │ "I'm not," he said.                          │                    │
│                                             │                                              │                    │
│                                             │ "I'm not," she said.                         │                    │
│                                             │                                              │                    │
│                                             │ "                                            │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Daniel flashed one of his own-a real one,   │  I was in the middle of the room. I was in   │ 3.5819809436798096 │
│ this time.                                  │ the middle of the room. I was in the middle  │                    │
│                                             │ of the room. I was in the middle of the      │                    │
│ "I almost had a heart attack when Mom       │ room.                                        │                    │
│ almost had a heart attack," he said, his    │                                              │                    │
│ voice quiet. Serious. "I'm-I'm happy you're │ "I'm sor